<a href="https://colab.research.google.com/github/ChickenRob0t/Proyecto-Covid-Meteorologia/blob/main/S_S_Fail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *Este fue el primer código que generé en el cual no funcionó la idea de implementación que tenía para segmentar los casos de covid por estaciones, sin embargo aprendí mucho cx*

In [ ]:
!pip install geopandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

# **Carga de archivos**

In [ ]:
UAG_2021=pd.read_csv("/content/drive/MyDrive/Datos S.S/Data/UAG/uag_2020_2021.txt", sep='\t')
Tet_20=pd.read_csv("/content/drive/MyDrive/Datos S.S/Data/Tetlan/tetlan_2020.txt", sep='\t')
Prep_2021=pd.read_csv("/content/drive/MyDrive/Datos S.S/Data/Prepa20/prepa20_2020_2021.txt", sep='\t')
Pira_tod=pd.read_csv("/content/drive/MyDrive/Datos S.S/Data/Pirámides/P_Sol_todos.txt", sep='\t')
Moli_2021=pd.read_csv("/content/drive/MyDrive/Datos S.S/Data/Molinos/molinos_2020_2021.txt", sep='\t')
Iam_2021=pd.read_csv("/content/drive/MyDrive/Datos S.S/Data/IAM/iam_2020_2021.txt", sep='\t')
Chiv_2021=pd.read_csv("/content/drive/MyDrive/Datos S.S/Data/Chivas/chivas_2020_2021.txt", sep='\t')
#Conag=pd.read_csv("/content/drive/MyDrive/Datos S.S/Data/Conagua/Datos CONAGUA_2019-2022.xlsx", sep='\t')

Covi=pd.read_excel("/content/drive/MyDrive/Datos S.S/Data/Base_datos_COVID2020_2022 (1).xlsx")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,9,10,16,17,18,27,28,29,30,31,32,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,9,10,17,18,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,3,4,5,6,7,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  war

 ## *Filtrado de datos de las columnas que consideré importantes (COVID)*

In [ ]:
col_i=['FECHREG','LOCRESI','LATLOCA','LONGLOCA','CP','RESDEFIN']
Import=Covi.loc[:,col_i]


In [ ]:
Import['RESDEFIN'].value_counts() #Para ver cuáles hay, y así poder filtrar sólo los casos de COVID

NEGATIVO                        5840
SARS-CoV-2                      4091
RECHAZADA                        243
SARS-CoV-2 - DELTA                36
VSR                               27
INF AH1N1 PMD                     24
ENTEROV//RHINOVIRUS               24
A H3                              13
METAPNEUMOVIRUS                   12
SARS-CoV-2 - OMICRON               9
B                                  7
VIRUS SINCICIAL RESPIRATORIO       7
SARS-CoV-2 - B.1.1.519             6
NO RECIBIDA                        6
CORONA NL63                        5
CORONA HKU1                        4
SARS-CoV-2 - GAMMA                 3
ADENOVIRUS                         3
SARS-CoV-2 - B.1.427/B.1.429       2
SARS-CoV-2 - ALPHA                 2
SARS-CoV-2 - OTRA                  2
BOCAVIRUS                          2
INF A                              2
NO ADECUADO                        2
PARAINFLUENZA 3                    1
SARS-CoV-2 - IOTA                  1
NO SUBTIPIFICADO                   1
C

In [ ]:
confirmados=Import.loc[Import['RESDEFIN'].str.contains('SARS') | Import['RESDEFIN'].str.contains('CORONA') ] #Sólo los que tienen covid

In [ ]:
#Guardo las primeras 40 localidades que tengan más personas
Localidades=confirmados['LOCRESI'].value_counts()[0:40].index.tolist()


In [ ]:
#Con este ciclo for si es NaN, '-' o '0.0' el CP,  intenta cambiarlos por el valor que más se repita de su localidad
for i in Localidades:


  max=confirmados.loc[confirmados['LOCRESI']== i]['CP'].value_counts().idxmax()

  if max in(['-',0.0]):
    confirmados['CP'].loc[(confirmados['LOCRESI']== i) & confirmados['CP'].isin ([np.nan,'-']) ]='pass'
    continue

  confirmados['CP'].loc[(confirmados['LOCRESI']== i) & confirmados['CP'].isin ([np.nan,'-',0.0]) ]=max

  #Con esto pasamos de tener 922 valores sin conocer a únicamente tener 382

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
#Ya que son únicamente 382, eliminamos esos datos
confirmados.iloc[confirmados['CP'].isin(['pass',0.0,'SD','-','.'])]=np.nan
confirmados=confirmados.dropna(subset=['CP'])
#Ahora ya tenemos todos los valores del CP, podemos pasar a encontrar sus valores de la latitud y longitud
confirmados.loc[confirmados['CP']=='4433O']=4430 #Problemilla de más delante

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## **Encontrando latitud y longitud**

Primero se intentó sacar a través de los códigos postales (los cuales eran muchísimos)

In [ ]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("44790, Jalisco")
location[1]

/usr/local/lib/python3.8/dist-packages/geopy/geocoders/osm.py:132: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  warnings.warn(


(20.666397556666666, -103.29687274)

In [ ]:
def code(cp):
  location = geolocator.geocode(str(cp) + ", Jalisco")
  place= geolocator.geocode(str(cp) + ", Jalisco")

  if place==None:
    return (0,0)
  else:
    return place[1]
  #Función que entrega el lugar en Jalisco con el código postal

In [ ]:
confirmados['CP']=confirmados['CP'].apply(lambda x: int(float(x)))  #Aquí, le quitamos los puntos decimales a los códigos postales para que agarren bien

In [ ]:
cps=confirmados['CP'].unique().tolist()  #Ponemos en una lista todos los códigos postales que encontramos y generamos un map con la función de arriba
la_lo=map(code,cps)

In [ ]:
lo_la=list(la_lo)
#Parece que dura muchísimo, podría ser que mejor lo intente de otra manera cx

Tardaba muchísimo, mejor decidí optar por sacar los CP a partir de los nombres de las localidades (que son sólo 150), así el programa tardaba menos en encontrarlas

In [ ]:
cp_loc=confirmados['LOCRESI'].unique().tolist()
map_lo=map(code,cp_loc)

In [ ]:
loc_list=list(map_lo)

In [ ]:
loc_list.count((0,0))

27

In [ ]:
#Diccionario con cada código postal y su latitud y longitud
LL_D=dict(zip(cp_loc,loc_list))

In [ ]:
LL_D={'GUADALAJARA': (20.6720375, -103.338396),
 'ACATLAN DE JUAREZ': (20.423970500000003, -103.6013031524341),
 'ZAPOPAN': (20.7211203, -103.3913671),
 'TEQUILA': (20.8821155, -103.8370907),
 'YAHUALICA DE GONZALEZ GALLO': (21.123410999999997, -102.95137191169567),
 'CUQUIO': (20.94949235, -103.03142458044168),
 'IXTLAHUACAN DE LOS MEMBRILLOS': (20.40302745, -103.20504879808468),
 'CIUDAD GUZMAN': (19.7001632, -103.463935),
 'TONALA': (20.6185208, -103.22273591034522),
 'PUERTO VALLARTA': (20.6407176, -105.220306),
 'SANTA CRUZ DE LAS FLORES': (20.3938983, -103.9492395),
 'TLAJOMULCO DE ZU?IGA': (0, 0),
 'TLAQUEPAQUE': (20.6397718, -103.3120428),
 'SALINAS DE HIDALGO': (22.639352, -101.7017067),
 'GRANJA SIN NOMBRE': (0, 0),
 'ARANDAS': (20.74931205, -102.3142822111025),
 'NEXTIPAC': (20.767103, -103.5263514),
 'EL SALTO': (20.539100050000002, -103.24037752852841),
 'COMPOSTELA': (20.6614803, -105.2302037),
 'ORILLA DEL RIO': (20.586683700000002, -103.13455412949347),
 'SAN IGNACIO CERRO GORDO': (20.76868135, -102.51481451130394),
 'CHAPALA': (20.279764450000002, -103.22035855623801),
 'DEGOLLADO': (20.45629675, -102.17482347106542),
 'VILLA ALBERTO ANDRES ALVARADO ARAMBURO': (0, 0),
 'TEOCALTICHE': (21.4816157, -102.52011498971609),
 'ACATIC': (20.76878345, -102.92039700254298),
 'PUERTA DE PERICOS (TLACHICHILCO)': (0, 0),
 'LA BARCA': (20.35720795, -102.50327003201096),
 'ZAPOTLANEJO': (20.6396569, -103.05151855846849),
 'LEON DE LOS ALDAMA': (0, 0),
 'AMECA': (20.5347004, -104.06466098593745),
 'SANTA ANITA': (20.5509918, -103.4437644),
 'TALA': (20.610062499999998, -103.70750330271915),
 'SAN MARTIN HIDALGO': (20.4357022, -103.9301557),
 'ACAPONETA': (20.7944341, -103.4914926),
 'OCOTEPEC': (0, 0),
 'MOROLEON': (20.5064341, -103.3650489),
 'OCOTLAN': (20.3872033, -102.73361924832378),
 'COCULA': (20.4035811, -103.81742103058032),
 'JOCOTEPEC': (20.289346950000002, -103.43686457681868),
 'IXTLAHUACAN DEL RIO': (20.904594250000002, -103.24987805369719),
 'SAYULA': (19.857163800000002, -103.60889569436725),
 'SAN SEBASTIAN DEL SUR': (19.7935613, -103.4749503),
 'ETZATLAN': (20.761220899999998, -104.13063108161292),
 'LOS GAVILANES': (20.556930299999998, -103.35840323683036),
 'MAGDALENA': (20.91062025, -104.11585399809384),
 'VALLE DE JUAREZ': (19.8146914, -102.98363356723414),
 'LAS BOVEDAS': (20.7349602, -103.424084),
 'AHUALULCO DE MERCADO': (20.710353750000003, -103.97539438473574),
 'VILLA CORONA': (20.38914285, -103.68552715478332),
 'TEUCHITLAN': (20.6806057, -103.83278673516206),
 'CHIMALTITAN': (21.741153699999998, -103.67909826087623),
 'TEPATITLAN DE MORELOS': (20.817055449999998, -102.6781613948367),
 'LA CALERA': (20.45051, -103.2533617),
 'TAPACHULA DE CORDOVA Y ORDO?EZ': (0, 0),
 'SAN CRISTOBAL DE LA BARRANCA': (21.046335650000003, -103.51239235477239),
 'GUANAJUATO': (20.7304835, -101.3739588),
 'COLOTLAN': (22.098424299999998, -103.27431707479644),
 'SAN PEDRO ITZICAN': (20.3156903, -102.9517846),
 'TAMAZULA DE GORDIANO': (19.6853653, -103.17006701011542),
 'SAN JUAN DE LOS LAGOS': (21.254312650000003, -102.28137105849761),
 'EL GRULLO': (19.79554805, -104.17781649818409),
 'SAN AGUSTIN': (20.5463888, -103.4668635),
 'TUXPAN': (19.4374898, -103.41147046920722),
 'ATEQUIZA': (20.3931514, -103.1385074),
 'UNION DE TULA': (20.004524349999997, -104.22767687376117),
 'BELLAVISTA': (20.4366325, -103.6347956),
 'SAN MIGUEL CUYUTLAN': (20.4170786, -103.3905524),
 'MANZANILLO': (20.5099369, -103.2973653),
 'LA MOJONERA': (20.3300424, -102.7890261),
 'NUEVO CASAS GRANDES': (0, 0),
 'TLAXCALA DE XICOHTENCATL': (0, 0),
 'ATLACOMULCO DE FABELA': (0, 0),
 'FRESNILLO': (20.588381, -103.3381413),
 'SANTA CRUZ AMILPAS': (0, 0),
 'CENTRO HIDROPONICO GOMEZ FARIAS': (0, 0),
 'ATOTONILCO EL ALTO': (20.5413692, -102.517566367728),
 'VILLA GUERRERO': (22.0193796, -103.67905169408235),
 'LA RIBERA': (20.3494738, -102.2826117),
 'MAZAMITLA': (19.890584150000002, -103.08102044422532),
 'TAPALPA': (19.946663450000003, -103.77610588768835),
 'VILLA DE GARCIA MARQUEZ (EL TARENGO NUEVO)': (0, 0),
 'SAN MARTIN DE LAS FLORES DE ABAJO': (0, 0),
 'AMATITAN': (20.844348699999998, -103.71051993442961),
 'EL VADO': (20.6091291, -103.155824),
 'AYUTLA': (20.04062435, -104.47395781043053),
 'BETANIA': (20.5806449, -102.3792814),
 'JESUS MARIA': (20.64794705, -102.10757376228773),
 'OJUELOS DE JALISCO': (21.786351449999998, -101.67889736335897),
 'JUANACATLAN': (20.485749, -103.15072540651236),
 'ABERRUSCO': (0, 0),
 4430: (20.623111, -103.3559123),
 'BENITO JUAREZ': (19.2199014, -102.6780356),
 'AHUACATLAN': (20.6092471, -103.3532998),
 'VERACRUZ': (20.6532095, -105.2218343),
 'QUITUPAN': (19.7944045, -102.88716124937912),
 'VISTAS III': (0, 0),
 'LAS ANIMAS DE QUIMIXTO': (0, 0),
 'TIZAPAN EL ALTO': (20.10729135, -103.09838165939337),
 'SAN NICOLAS DE IBARRA': (20.3289105, -103.1288962),
 'LOS FRESEROS': (0, 0),
 'LA ESPERANZA': (20.8799903, -102.7692794),
 'SAN VICENTE': (20.4282709, -102.6544245),
 'TECOLOTLAN': (20.275808599999998, -103.98225523369862),
 'ZACOALCO DE TORRES': (20.244094949999997, -103.59310626655152),
 'JALOSTOTITLAN': (21.201131750000002, -102.51069605240619),
 'PONCITLAN': (20.263965300000002, -102.88279855436468),
 'TANHUATO DE GUERRERO': (0, 0),
 'PAJACUARAN': (0, 0),
 'GUACHINANGO': (20.696709300000002, -104.41541268197736),
 'SANTA MARIA DE LA PAZ': (0, 0),
 'TALPA DE ALLENDE': (20.2986583, -104.9524361256174),
 'SANTA MARIA DE LOS ANGELES': (22.19050555, -103.18899010736042),
 'SAN JULIAN': (20.993872, -102.15835703145814),
 'AGUA PRIETA': (20.7923909, -102.891726),
 'CIUDAD MADERO': (20.61048, -103.28322),
 'MORELIA': (19.7114821, -101.1717083),
 'ZAPOTLAN DEL REY': (20.46680925, -102.92302911944518),
 'ZAPOTITLAN DE VADILLO': (19.503478700000002, -103.74883288487334),
 'CERRO GORDO': (20.7594946, -102.5891453),
 'ACAPULCO DE JUAREZ': (16.8646266, -99.901852),
 'AJIJIC': (20.3001977, -103.2619347),
 'XICOTEPEC DE JUAREZ': (0, 0),
 'LA HIGUERA': (19.4233927, -103.4098578),
 'SAN MIGUEL DE LA PAZ': (20.3702677, -102.6627865),
 'HOSTOTIPAQUILLO': (21.0584771, -104.050605),
 'AGUASCALIENTES': (21.3621377, -101.9409428),
 'AYOTLAN': (20.47223855, -102.3641292878145),
 'SAN JUANITO DE ESCOBEDO': (20.8136991, -103.9923066295169),
 'MEZQUITIC': (22.2313494, -104.1102847617064),
 'TEPAME': (20.7083627, -103.4468134),
 'JACONA DE PLANCARTE': (19.9480312, -102.3149795),
 'LA PAZ DE ORDAZ': (0, 0),
 'AUTLAN DE NAVARRO': (19.75703, -104.31834015548738),
 'CIHUATLAN': (19.263284849999998, -104.60497365939882),
 'TECAMAC DE FELIPE VILLANUEVA': (19.7182209, -98.9645163),
 'COLIMA': (20.6292273, -105.2164695),
 'VILLA PURIFICACION': (19.82320785, -104.73273779114446),
 'MI GRANJITA': (0, 0),
 'LA PAZ': (20.5396776, -103.0513766),
 'NAHUAPA': (19.9207681, -105.3183541),
 'LA PIEDRERA': (20.7756507, -103.3454871),
 'MASCOTA': (20.5533013, -104.76338164381983),
 'LA UNION': (19.9144306, -102.7901964),
 'SAN IGNACIO RIO MUERTO': (0, 0),
 'SAN MIGUEL EL ALTO': (20.99270025, -102.39359813114054),
 'APANGO': (0, 0),
 'SANTA GERTRUDIS (SANTA CRUZ)': (0, 0),
 'POTRERILLOS': (20.3380245, -103.3761834),
 'JAMAY': (20.293352650000003, -102.68652379075318)}

In [ ]:
faltan={'TLAJOMULCO DE ZU?IGA':(20.4739,-103.447),
'GRANJA SIN NOMBRE':(20.6794244, -103.4481621),
'VILLA ALBERTO ANDRES ALVARADO ARAMBURO':(27.644444,113.3847),
'PUERTA DE PERICOS (TLACHICHILCO)':(20.655556,-104.148333),
'LEON DE LOS ALDAMA':(21.12908,-101.67374),
'OCOTEPEC': (20.2847,-103.428),
'TAPACHULA DE CORDOVA Y ORDO?EZ':(14.908056,-92.260556),
'NUEVO CASAS GRANDES':(30.417,107.9125),
'TLAXCALA DE XICOHTENCATL':(19.3190500,-98.1998200),
'ATLACOMULCO DE FABELA':(19.797500,-99.874444),
'SANTA CRUZ AMILPAS':(17.058333,-96.683333),
'CENTRO HIDROPONICO GOMEZ FARIAS':(19.7929,-103.475 ),
'VILLA DE GARCIA MARQUEZ (EL TARENGO NUEVO)':(20.431111,-102.496389),
'SAN MARTIN DE LAS FLORES DE ABAJO':(20.585278,-103.282778),
'ABERRUSCO':(20.200556,-103.981389),
'VISTAS III':(20.7082042,-103.3619351),
'LAS ANIMAS DE QUIMIXTO':(20.506667,-105.344444),
'LOS FRESEROS':(20.419167,-103.188889),
'TANHUATO DE GUERRERO':(20.281944,-102.331944),
'PAJACUARAN':(20.1189,-102.566),
'SANTA MARIA DE LA PAZ':(21.5134,-103.408),
'XICOTEPEC DE JUAREZ':(20.274167,-97.955000),
'LA PAZ DE ORDAZ':(20.3333,-102.533),
'MI GRANJITA':(19.25465,-99.10356),
'SAN IGNACIO RIO MUERTO':(27.35362,-110.45337),
'APANGO':(19.781944, -103.711111),
'SANTA GERTRUDIS (SANTA CRUZ)':(22.07278,-104.19083),
}8

In [ ]:
for i in faltan:
  LL_D[i]=faltan[i]

#Con esto ya tenemos el diccionario con todas las localidades asignadas les tenemos una localización latitud y longitud

In [ ]:
confirmados=confirmados.reset_index()

In [ ]:
for i in LL_D:
  confirmados['LATLOCA'].loc[confirmados['LOCRESI']==i]=LL_D[i][0]

for i in LL_D:
  confirmados['LONGLOCA'].loc[confirmados['LOCRESI']==i]=LL_D[i][1]
#Así, ahora todas las locaciones tienen sus coordenadas bien definidas

In [ ]:
#Agregamos los puntos que serán el centro de cada una de las estaciones
#Points={'Guadalajara':(20.67425, -103.35006),'IAM' :(20.674609, -103.384118),'La Experiencia': (20.67011, -103.2855),'Molinos': (20.80566, -103.45767),'Pla_sol':(20.65511, -103.39836), 'Prepa 20': (20.63019, -103.46544),'Tlaquepaque': (20.63813, -103.30823),'Tonala': (20.62449, -103.24089), 'UAG': (20.68703, -103.39194),'Zapopan': (20.7252, -103.38905), 'Zapotlanejo': (20.61913, -103.06633)}
Points={'Pir_sol':(20.62638252600345, -103.43335530002135),'IAM' :(20.674609, -103.384118),'Guadalajara':(20.66682,-103.39182), 'Prepa 20': (20.63019, -103.46544), 'UAG': (20.68703, -103.39194),'Tetlan':(20.662216168559688, -103.27287521774362), 'Tlaquepaque':(20.64091,-103.29327), 'Zapopan':(20.7252, -103.38905), 'Zapotlanejo':(20.61913, -103.06633), 'Tonala':(20.62449, -103.24089),'Molinos': (20.80566, -103.45767),'Acatlán de Juárez':(20.4231,-103.58920)}

In [ ]:
Estaciones=[1,2,3,4,5,6,7,8,9,0,1,2]
Latitud=[1,2,3,4,5,6,7,8,9,0,1,2]
Longitud=[1,2,3,4,5,6,7,8,9,0,1,2]
s=0
for i in Points:
  Estaciones[s],Longitud[s],Latitud[s]= i ,Points[i][1],Points[i][0]
  s=s+1


In [ ]:
EstacionesD=pd.DataFrame({'Estaciones':Estaciones, 'Latitud': Latitud, 'Longitud': Longitud})


,Estaciones,Latitud,Longitud
0,Pir_sol,20.626383,-103.433355
1,IAM,20.674609,-103.384118
2,Guadalajara,20.666820,-103.391820
3,Prepa 20,20.630190,-103.465440
4,UAG,20.687030,-103.391940
5,Tetlan,20.662216,-103.272875
6,Tlaquepaque,20.640910,-103.293270
7,Zapopan,20.725200,-103.389050
8,Zapotlanejo,20.619130,-103.066330
9,Tonala,20.624490,-103.240890


## *Creando los mapas y filtrándolos**

In [ ]:
import folium
from folium import Marker, GeoJson
from folium.plugins import HeatMap

import geopandas as gpd
#Nota para mí, cambiar los epsg a ver qué pasa

In [ ]:
Estaciones5=EstacionesD.iloc[7:,:]
Estaciones2=EstacionesD.iloc[:7,:]
geo_confirmados=gpd.GeoDataFrame(confirmados,
                                 geometry=gpd.points_from_xy(confirmados.LONGLOCA,confirmados.LATLOCA),
                                 crs='epsg:4326')

geo_stations_5=gpd.GeoDataFrame(Estaciones5,
                                geometry=gpd.points_from_xy(Estaciones5.Longitud,Estaciones5.Latitud),
                                crs='epsg:4326')


geo_stations_2=gpd.GeoDataFrame(Estaciones2,
                                geometry=gpd.points_from_xy(Estaciones2.Longitud,Estaciones2.Latitud)
                                ,crs='epsg:4326')
Estaciones5

,Estaciones,Latitud,Longitud,geometry
7,Zapopan,20.72520,-103.38905,POINT (-103.38905 20.72520)
8,Zapotlanejo,20.61913,-103.06633,POINT (-103.06633 20.61913)
9,Tonala,20.62449,-103.24089,POINT (-103.24089 20.62449)
10,Molinos,20.80566,-103.45767,POINT (-103.45767 20.80566)
11,Acatlán de Juárez,20.42310,-103.58920,POINT (-103.58920 20.42310)


In [ ]:
print(geo_stations_5.crs)
print(geo_stations_2.crs)
print(geo_confirmados.crs)


EPSG:32634
EPSG:32634
epsg:4326


0

In [ ]:
geo_stations_5= geo_stations_5.to_crs("EPSG:32634") #Cambios el EPSG para que funcione usar el buffer
geo_stations_2=geo_stations_2.to_crs("EPSG:32634")


dos_km=geo_stations_2['geometry'].buffer(2000) #Generamos círculos de 2 y 5 km para cada estación
cin_km=geo_stations_5['geometry'].buffer(5000)


In [ ]:
m=folium.Map(location=[20.674609,-103.384118],zoom_start=12)
HeatMap(data=confirmados[['LATLOCA', 'LONGLOCA']], radius=15).add_to(m)
for idx, row in EstacionesD.iterrows():
  Marker([row['Latitud'],row['Longitud']],popup='Estación: {} <br> Latitud: {} <br> Longitud {}'.format(EstacionesD['Estaciones'][idx], EstacionesD['Latitud'][idx],EstacionesD['Longitud'][idx]),radius=20).add_to(m)

GeoJson(dos_km).add_to(m)
GeoJson(cin_km).add_to(m)
m

## Intentaré hacerlo sumando lo que hizo Gudiño, con alguna información de LOCRESI

In [ ]:
confirmados_2=confirmados  #Hacemos una copia para ver si funciona, si funciona lo aplicamos a todo jeje

In [ ]:
confirmados_2['ESTACIONES']=np.zeros(len(confirmados_2))


confirmados_2['ESTACIONES'].loc[confirmados_2['LOCRESI']=='ZAPOPAN']='Zapopan'
confirmados_2['ESTACIONES'].loc[confirmados_2['LOCRESI']=='ZAPOTLANEJO']='Zapotlanejo'
confirmados_2['ESTACIONES'].loc[confirmados_2['LOCRESI']=='TEPATITLAN DE MORELOS']='Tepatitlan'
confirmados_2['ESTACIONES'].loc[confirmados_2['LOCRESI']=='TLAJOMULCO DE ZU?IGA']='Prepa 20'
confirmados_2['ESTACIONES'].loc[confirmados_2['LOCRESI']=='TONALA']='Tonala'
confirmados_2['ESTACIONES'].loc[confirmados_2['LOCRESI']=='TLAQUEPAQUE']='Tlaquepaque'
confirmados_2

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,FECHREG,LOCRESI,LATLOCA,LONGLOCA,CP,RESDEFIN,ESTACIONES
3,2020-06-16 00:00:00,GUADALAJARA,20.676389,NaN,45509,SARS-CoV-2,0.0
8,2020-12-16 00:00:00,GUADALAJARA,20.676389,NaN,44320,SARS-CoV-2,0.0
12,2020-09-15 00:00:00,ACATLAN DE JUAREZ,20.420556,NaN,45700,SARS-CoV-2,0.0
13,2020-07-21 00:00:00,GUADALAJARA,20.676389,NaN,44790,SARS-CoV-2,0.0
14,2020-07-21 00:00:00,GUADALAJARA,20.676389,NaN,44790,SARS-CoV-2,0.0
...,...,...,...,...,...,...,...
11856,2022-01-16 00:00:00,GUADALAJARA,20.676389,NaN,45079,SARS-CoV-2,0.0
11860,2022-02-12 00:00:00,ZAPOPAN,20.720278,NaN,45100,SARS-CoV-2,Zapopan
11870,2022-01-24 00:00:00,SANTA CRUZ DE LAS FLORES,20.480000,NaN,45640,SARS-CoV-2,0.0
11872,2022-02-08 00:00:00,GUADALAJARA,20.676389,NaN,44730,SARS-CoV-2,0.0


In [ ]:
(confirmados_2['ESTACIONES']==0.0).sum() #Hay todavía 1513 casos que no sabemos en qué estación están cx
#Quizás podría encontrar con este método junto con el método de Gudiño para ver qué tantos puedo rescatar
#Otra idea que tengo es encontrar los

1513

In [ ]:
CPS=pd.read_csv('/content/drive/MyDrive/Datos S.S/Data/PP.csv')

In [ ]:
CPS=CPS.rename(columns={CPS.columns[0]:'CP', CPS.columns[1]:'Estacion'})

In [ ]:
for i in range(len(CPS)):
  confirmados_2['ESTACIONES'].loc[ confirmados_2['CP']==CPS['CP'][i] ]=CPS['Estacion'][i]


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
Casos_N=(confirmados_2['ESTACIONES']==0).sum() #Ahora sólo quedan 936 personas positivas que no sabemos a qué estación pertenecen

In [ ]:
len(confirmados_2)-Casos_N  #2839 datos fueron clasificados en las estaciones, creo que es lo mejor que he logrado

2839

In [ ]:
confirmados_2.loc[confirmados_2['ESTACIONES']==0]

NameError: ignored